In [11]:
import arcpy
folder = r'E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Assignment 5\Assignment5'
arcpy.env.workspace = folder
arcpy.arcpy.env.overwriteOutput = True

In [5]:
fcList = arcpy.ListFeatureClasses()
for fc in fcList:
    print(fc)

census2010_counties
eu_cities
north_america_rivers
states48_albers


## Question 1

In [57]:
###################################################################### 
# Problem 1 (30 Points)
#
# Given a polygon feature class in a geodatabase, a count attribute of the feature class(e.g., population, disease count):
# this function calculates and appends a new density column to the input feature class in a geodatabase.

# Given any polygon feature class in the geodatabase and a count variable:
# - Calculate the area of each polygon in square miles and append to a new column
# - Create a field (e.g., density_sqm) and calculate the density of the selected count variable
#   using the area of each polygon and its count variable(e.g., population) 
# 
# 1- Check whether the input variables are correct(e.g., the shape type, attribute name)
# 2- Make sure overwrite is enabled if the field name already exists.
# 3- Identify the input coordinate systems unit of measurement (e.g., meters, feet) for an accurate area calculation and conversion
# 4- Give a warning message if the projection is a geographic projection(e.g., WGS84, NAD83).
#    Remember that area calculations are not accurate in geographic coordinate systems. 
# 
###################################################################### 
def calculateDensity(fcpolygon, attribute, geodatabase = "assignment2.gdb"):
    try:
        ## overwrite enabled, update gdb
        arcpy.arcpy.env.overwriteOutput = True
        arcpy.env.workspace = folder + geodatabase
        print("overwrite enabled and gdb updated")
        
        area_field = 'area_sqmi'
        density_field = 'density_sqm'
        
        ## adding new fields
        arcpy.AddField_management(fcpolygon, area_field, "DOUBLE")
        arcpy.AddField_management(fcpolygon, density_field, "DOUBLE")
        print("new columns created")
        
        ## format the expression
        expression = '"!'+ attribute + '!/!' + area_field + '!"'
        print('Expression in use: ' + expression)

        ## calculate area in sqmi
        arcpy.CalculateGeometryAttributes_management(fc, [[area_field, "AREA_GEODESIC"]], 'MILES_US', "SQUARE_MILES_US")
        print('area calculated')
        
        ## calculate density
        arcpy.CalculateField_management(fcpolygon, density_field, expression, "PYTHON3")
        print('density calcaulated')
    
    except:
        # By default any other errors will be caught here
        e = sys.exc_info()[1]
        print(e.args[0])

In [39]:
attribute = 'POPULATION'
area_field = 'area_sqmi'
density_field = 'density_sqm'
expression = '"!'+ attribute + '!/!' + area_field + '!"'
print(expression)

"!POPULATION!/!area_sqmi!"


In [40]:
fcpolygon = 'states48_albers'
fcpolygon

'states48_albers'

In [59]:
calculateDensity(fcpolygon, attribute, 'hw5.gdb')

overwrite enabled and gdb updated
new columns created
Expression in use: "!POPULATION!/!area_sqmi!"
area calculated
density calcaulated


### Testing

In [56]:
arcpy.CalculateField_management(fcpolygon, density_field, expression, "PYTHON3")

<Result 'states48_albers'>

## Question 2

In [78]:
###################################################################### 
# Problem 2 (40 Points)
# 
# Given a line feature class (e.g.,river_network.shp) and a polygon feature class (e.g.,states.shp) in a geodatabase, 
# id or name field that could uniquely identify a feature in the polygon feature class
# and the value of the id field to select a polygon (e.g., Iowa) for using as a clip feature:
# this function clips the linear feature class by the selected polygon boundary,
# and then calculates and returns the total length of the line features (e.g., rivers) in miles for the selected polygon.
# 
# 1- Check whether the input variables are correct (e.g., the shape types and the name or id of the selected polygon)
# 2- Transform the projection of one to other if the line and polygon shapefiles have different projections
# 3- Identify the input coordinate systems unit of measurement (e.g., meters, feet) for an accurate distance calculation and conversion
#        
###################################################################### 
def estimateTotalLineLengthInPolygons(fcLine, fcClipPolygon, polygonIDFieldName, clipPolygonID, geodatabase = "assignment2.gdb"):
    ## overwrite enabled, update gdb
    arcpy.arcpy.env.overwriteOutput = True
    arcpy.env.workspace = folder + geodatabase
    print("overwrite enabled and gdb updated")
    
    lengthField = 'length_mi'
    lengthTotal = 'length_total'
    outName = 'line_overlap'
    
     ## add length field to polyogn
    arcpy.AddField_management(fcClipPolygon, lengthField, "DOUBLE")
    print("Length Field added")
    
    ## Intersect Analysis
    arcpy.Intersect_analysis([fcClipPolygon, fcLine], outName)
    print('Intersect Analysis complete.')
    
    ## Length in each
    arcpy.CalculateGeometryAttributes_management(outName, [[lengthField, "LENGTH_GEODESIC"]], "MILES_US")
    print('Length calculated')
    
    polyDict = dict()
    with arcpy.da.SearchCursor(outName, [polygonIDFieldName, lengthField]) as cursor:
        for row in cursor:
            id = row[0]
            if polygonIDFieldName in polyDict.keys():
                polyDict[id] += row[1]
            else:
                polyDict[id] = row[1]
    
    with arcpy.da.UpdateCursor(fcClipPolygon, [polygonIDFieldName, lengthField]) as cursor:
        for row in cursor:
            if row[0] in polyDict.keys():
                row[1] = polyDict[row[0]]
            else:
                row[1] = 0
            cursor.updateRow(row)

In [79]:
fcLine = 'north_america_rivers'
fcClipPolygon = 'census2010_counties'
polygonIDFieldName = 'GEOID10'
clipPolygonID = 'GEOID10'

In [80]:
estimateTotalLineLengthInPolygons(fcLine, fcClipPolygon, polygonIDFieldName, clipPolygonID, 'hw5.gdb')

overwrite enabled and gdb updated
Length Field added
Intersect Analysis complete.
Length calculated


### Testing

In [72]:
lengthField = 'length_mi'
lengthTotal = 'length_total'
outName = 'line_overlap'

In [73]:
 ## add length field to polyogn
arcpy.AddField_management(fcClipPolygon, lengthField, "DOUBLE")
print("Length Field added")

Length Field added


In [74]:
## Intersect Analysis
arcpy.Intersect_analysis([fcClipPolygon, fcLine], outName)
print('Intersect Analysis complete.')

Intersect Analysis complete.


In [75]:
## Length in each
arcpy.CalculateGeometryAttributes_management(outName, [[lengthField, "LENGTH_GEODESIC"]], "MILES_US")
print('Length calculated')

Length calculated


In [76]:
polyDict = dict()
with arcpy.da.SearchCursor(outName, [polygonIDFieldName, lengthField]) as cursor:
    for row in cursor:
        id = row[0]
        if polygonIDFieldName in polyDict.keys():
            polyDict[id] += row[1]
        else:
            polyDict[id] = row[1]
polyDict

{'19063': 1.7649323513219006, '19119': 0.8663990822790417, '19143': 1.7456229545505342, '19189': 3.310783818424677, '19089': 3.1211527309500386, '19191': 2.3661065729007396, '19059': 2.476758736155351, '19109': 0.3803148376165462, '19195': 0.5980255400269998, '19131': 0.33402050767068664, '19005': 0.5000328893874033, '19167': 16.05037765833247, '19041': 4.790490575534031, '19081': 0.35779161388869046, '19033': 0.0644294842560822, '19141': 2.9076202516701106, '19147': 10.082284238191924, '19067': 7.3133793474886835, '19037': 1.1805542723665867, '19065': 2.4989940621935234, '19043': 1.1394701055887047, '19069': 1.031549048470551, '19023': 2.4316522038219834, '19017': 18.32462948777239, '19149': 9.143818807317158, '19035': 0.1177338457332449, '19021': 2.5638051221592817, '19091': 0.3623165249391645, '19151': 0.9298250713759743, '19197': 1.4982503345428768, '19061': 0.4418961215013761, '19013': 1.7212426543327783, '19055': 0.3514623817806275, '19019': 0.24618858718986497, '19187': 3.911050

In [77]:
with arcpy.da.UpdateCursor(fcClipPolygon, [polygonIDFieldName, lengthField]) as cursor:
    for row in cursor:
        if row[0] in polyDict.keys():
            row[1] = polyDict[row[0]]
        else:
            row[1] = 0
        cursor.updateRow(row)

## Question 3

In [ ]:
######################################################################
# Problem 3 (30 points)
# 
# Given an input point feature class, (i.e., eu_cities.shp) and a distance threshold and unit:
# Calculate the number of points within the distance threshold from each point (e.g., city),
# and append the count to a new field (attribute).
#
# 1- Identify the input coordinate systems unit of measurement (e.g., meters, feet, degrees) for an accurate distance calculation and conversion
# 2- If the coordinate system is geographic (latitude and longitude degrees) then calculate bearing (great circle) distance
#
######################################################################
def countObservationsWithinDistance(fcPoint, distance, distanceUnit, geodatabase = "assignment2.gdb"):
    ## overwrite enabled, update gdb
    arcpy.arcpy.env.overwriteOutput = True
    arcpy.env.workspace = folder + geodatabase
    print("overwrite enabled and gdb updated")
    
    
    
    

In [88]:
fcPoint = 'eu_cities'
distance = 200
distanceUnit = 'MILES'

In [ ]:
# countObservationsWithinDistance(fcPoint, distance, distanceUnit, 'hw5.gdb')

### Testing

In [89]:
outFc = 'spatialJoin'
distanceCombined = str(distance) + " " + distanceUnit
distanceCombined

'200 MILES'

In [94]:
fieldMap = arcpy.FieldMap()
fieldMap.addFieldMap('Count')
fieldMap

AttributeError: 'FieldMap' object has no attribute 'addFieldMap'

In [90]:
## spatial join to acquire all within distances
arcpy.analysis.SpatialJoin(fcPoint, fcPoint, outFc, 'JOIN_ONE_TO_ONE', '#', '#', 'WITHIN_A_DISTANCE_GEODESIC', distanceCombined)

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Assignment 5\\Assignment5\\Assignment5.gdb\\spatialJoin'>

In [ ]:
outSummary = 'countSummary'
arcpy.analaysis.Statistics(outFc, outSummary, [['COUNT']], 'ObjectID')

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#